

> Constructing Node Features <br>
> fMRI feature vector



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
%matplotlib inline
import torch
import torch.nn as nn

In [9]:
#mount gdrive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
# Paths
fMRI_BOLD_data_dir = "/content/drive/MyDrive/UofSC/AIISC/Data-fMRI/NL_Deep_fMRI/"


SUB_IDs = ["UTS01","UTS03","UTS07"]

# /content/gdrive/MyDrive/UofSC/AIISC/Data-fMRI/NL_Deep_fMRI/UTS01_combined.npy
fMRI_data = np.load(os.path.join(fMRI_BOLD_data_dir, f"{SUB_IDs[0]}_combined.npy"))
print(fMRI_data.shape)

(8974, 423)


adollshouse: TRs = 241, range = 5–245<br>
adventuresinsayingyes: TRs = 391, range = 5–395<br>
alternateithicatom: TRs = 343, range = 5–347<br>
avatar: TRs = 366, range = 5–370<br>
buck: TRs = 332, range = 5–336<br>
exorcism: TRs = 466, range = 5–470<br>
eyespy: TRs = 379, range = 5–383<br>
hangtime: TRs = 324, range = 5–328<br>
haveyoumethimyet: TRs = 494, range = 7–500<br>
howtodraw: TRs = 353, range = 5–357<br>
inamoment: TRs = 205, range = 5–209<br>
itsabox: TRs = 355, range = 5–359<br>
legacy: TRs = 400, range = 5–404<br>
myfirstdaywiththeyankees: TRs = 358, range = 5–362<br>
naked: TRs = 422, range = 5–426<br>
odetostepfather: TRs = 404, range = 5–408<br>
sloth: TRs = 437, range = 5–441<br>
souls: TRs = 355, range = 5–359<br>
stagefright: TRs = 293, range = 5–297<br>
swimmingwithastronauts: TRs = 385, range = 5–389<br>
thatthingonmyarm: TRs = 434, range = 5–438<br>
theclosetthatateeverything: TRs = 314, range = 5–318<br>
tildeath: TRs = 323, range = 5–327<br>
undertheinfluence: TRs = 303, range = 5–307<br>
wheretheressmoke: TRs = 291, range = 5–295<br>


1.   A notable discrepency, each subjects' fMRI volume has 8974 TRs. However, stimuli converted from seconds to TR has a volume of 8968 TRs.
2.   After rechecking exact borders of stimuli and fMRI cutoffs, we identified these missing TRs in the stimuli TRs which we need to manually remove from subjects' fMRI data.

'avatar' remove last TR <br>
'exorcism' remove last TR<br>
'haveyoumethimyet' remove first two TRs<br>
'howtodraw' remove last TR<br>
'undertheinfluence' remove last TR<br>


In [11]:
# I need to remove six TRs
TRs_to_remove = [1340, 2138, 2842, 2843, 3688, 8676]
fmri_fixed = np.delete(fMRI_data, TRs_to_remove, axis=0)
print(fmri_fixed.shape)   # (8968, 423)

(8968, 423)


# Loading FastText dataframe

In [15]:
df = pd.read_csv('/content/drive/MyDrive/UofSC/AIISC/Data-fMRI/codes/fasttext/Huth_fMRI_25_ordered_stories_SCOPEmerged_Spacy_tagged_trimmed_fasttext_df_TR-levelFinal.csv')
display(df)

,story,TR,embedding
0,adollshouse,5,[ 2.91058570e-02 7.36758206e-03 -4.48260605e-...
1,adollshouse,6,[ 4.85898145e-02 -3.82069051e-02 5.44266263e-...
2,adollshouse,7,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...
3,adollshouse,8,[ 9.60598439e-02 -2.50132322e-01 -9.59771574e-...
4,adollshouse,9,[ 3.90336700e-02 -1.70992240e-02 7.91714247e-...
...,...,...,...
8963,wheretheressmoke,291,[ 5.64981103e-02 -9.63317081e-02 -3.38661335e-...
8964,wheretheressmoke,292,[-4.66044154e-03 1.46270944e-02 4.89505380e-...
8965,wheretheressmoke,293,[ 3.20332944e-02 7.78971845e-03 5.17318062e-...
8966,wheretheressmoke,294,[ 6.26441613e-02 -1.67448655e-01 -7.55591616e-...


In [18]:
type(df.at[0, 'embedding'])

str

In [20]:
# Function to fix the stringified embedding column
def clean_embedding_string(x):
    """
    Parses a string representation of a numpy array back into a numpy array.
    Handles both comma-separated '[0.1, 0.2]' and space-separated '[0.1 0.2]'
    """
    if isinstance(x, str):
        # Remove brackets and newlines
        clean_str = x.replace('[', '').replace(']', '').replace('\n', ' ')

        # Determine separator (comma or space)
        if ',' in clean_str:
            return np.fromstring(clean_str, sep=',')
        else:
            return np.fromstring(clean_str, sep=' ')
    return x

# Apply the cleaning
print("Parsing embedding strings...")
df['embedding'] = df['embedding'].apply(clean_embedding_string)

# Verify it worked
print(f"Type after fix: {type(df.at[0, 'embedding'])}") # Should be <class 'numpy.ndarray'>
print(f"Shape of one embedding: {df.at[0, 'embedding'].shape}") # Should be (300,)

# Convert all embeddings to a single Torch Tensor for speed
# Shape: (8968, 300)
word_data_tensor = torch.tensor(np.stack(df['embedding'].values), dtype=torch.float32)

Parsing embedding strings...
Type after fix: <class 'numpy.ndarray'>
Shape of one embedding: (300,)


In [19]:
# Constants based on your data
NUM_TRS = 8968
NUM_NODES = 423
WINDOW_SIZE = 20  # Look back 20 TRs
D_FMRI = 32       # Desired size of fMRI embedding
D_WORD = 300      # Size of BERT/Word2Vec embedding

In [6]:
# 1. The Encoder Model (Example: Simple 1D CNN)
class TimeSeriesEncoder(nn.Module):
    def __init__(self):
        super().__init__()
        # Input channels=1 (raw BOLD), Output channels=16
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=16, kernel_size=5)
        self.fc = nn.Linear(16 * (WINDOW_SIZE - 4), D_FMRI) # Flatten and project

    def forward(self, x):
        # x shape: (Batch_Nodes, 1, Window_Size)
        x = self.conv1(x)
        x = torch.relu(x)
        x = x.view(x.size(0), -1) # Flatten
        x = self.fc(x)
        return x # Shape: (Batch_Nodes, D_FMRI)

encoder = TimeSeriesEncoder()

In [ ]:
# --- 3. MAIN LOOP ---

feature_list = []

print("Starting Loop...")

for t in range(NUM_TRS):

    # --- A. PREPARE FMRI WINDOW ---
    if t < WINDOW_SIZE:
        # Padding for the start of the story
        window = torch.zeros(NUM_NODES, 1, WINDOW_SIZE)
    else:
        # Get window: (WINDOW_SIZE, 423)
        raw_slice = fmri_fixed[t-WINDOW_SIZE : t, :]

        # Transpose to (423, WINDOW_SIZE)
        raw_slice_T = raw_slice.T

        # Convert to Tensor and add Channel Dim -> (423, 1, WINDOW_SIZE)
        window = torch.tensor(raw_slice_T, dtype=torch.float32).unsqueeze(1)

    # --- B. ENCODE FMRI ---
    # Result shape: (423, 32)
    # Note: Using torch.no_grad() is faster if you aren't training the encoder right now
    with torch.no_grad():
        fmri_embedding = encoder(window)

    # --- C. GET WORD FEATURES ---
    # Grab the pre-processed tensor for time t
    word_vec = word_data_tensor[t] # Shape: (300,)

    # Broadcast/Repeat: (300,) -> (423, 300)
    word_expanded = word_vec.repeat(NUM_NODES, 1)

    # --- D. CONCATENATE ---
    # (423, 32) + (423, 300) -> (423, 332)
    node_features_t = torch.cat([fmri_embedding, word_expanded], dim=1)

    feature_list.append(node_features_t)

# --- 4. FINALIZE ---
final_X = torch.stack(feature_list)
print("Final X Shape:", final_X.shape)
# Expected: (8968, 423, 332)

Starting Loop...
